<a href="https://colab.research.google.com/github/lucasbegue/clases-ML/blob/main/notebook_clase2_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [1]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

# Configuración matplotlib
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

# Datos

Descargar el archivo 'meatspec.csv' de la carpeta 'datasets' del drive. Despues subirlo a esta notebook.


El departamento de calidad de una empresa de alimentación se encarga de medir el contenido en grasa de la carne que comercializa. El proceso de medición de grasa es costoso en tiempo y recursos. Una alternativa que permitiría reducir costos y tiempo es emplear un espectrofotómetro (instrumento capaz de detectar la absorbancia que tiene un material a diferentes tipos de luz) e inferir el contenido en grasa a partir de sus medidas.

Antes de dar por válida esta nueva técnica, la empresa necesita comprobar qué margen de error tiene respecto al análisis químico. Para ello, se mide el espectro de absorbancia a 100 longitudes de onda en 215 muestras de carne, cuyo contenido en grasa se obtiene también por análisis químico, y se entrena un modelo con el objetivo de predecir el contenido en grasa a partir de los valores dados por el espectrofotómetro.

In [2]:
# Datos
df = pd.read_csv('/content/meatspec.csv')
df.shape

FileNotFoundError: ignored

In [ ]:
df.head()

# Relación entre variables

In [ ]:
corr_matrix = df.select_dtypes(include=['float64', 'int']).corr(method='pearson')

# Heatmap matriz de correlaciones
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.heatmap(
    corr_matrix, square=True, ax=ax
)
ax.tick_params(labelsize=3)

Vemos que las variables estan todas muy correlacionadas entre sí. En general se observan correlaciones absolutas >0.9, lo que implica un problema para modelos de regresión lineal. 

# Modelos

In [ ]:
# División de los datos en train y test
X = df.drop(columns='fat')
y = df['fat']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y.values.reshape(-1,1),
    train_size=0.7,
    random_state=23,
    shuffle=True
)

### Mínimos cuadrados (OLS)

In [ ]:
# Creación y entrenamiento del modelo
modelo = LinearRegression(normalize=True)
modelo.fit(X=X_train, y=y_train)
LinearRegression(normalize=True)

# Coeficientes del modelo
df_coeficientes = pd.DataFrame(
    {
     'predictor': ['intersecto'] + list(X_train.columns.values),
     'coef': [modelo.intercept_] + list(modelo.coef_.flatten())
    }
)

fig, ax = plt.subplots(figsize=(11, 3.84))
ax.stem(df_coeficientes.predictor, df_coeficientes.coef, markerfmt=' ')
plt.xticks(rotation=90, ha='right', size=5)
ax.set_xlabel('variable')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo')
plt.show()

In [ ]:
# Predicciones test
y_pred = modelo.predict(X=X_test)
y_pred = y_pred.flatten()

# Error de test del modelo 
rmse_ols = mean_squared_error(
    y_true  = y_test,
    y_pred  = y_pred,
    squared = False
)

print(f"El error (rmse) de test es: {rmse_ols}")

### Ridge

In [ ]:
# Creación y entrenamiento del modelo (con búsqueda por CV del valor óptimo alpha)
modelo = RidgeCV(
    alphas = np.logspace(-10, 2, 200), # prueba 200 valores en el intervalo [10^-10; 10^2]
    fit_intercept = True,
    normalize = True,
    store_cv_values = True
)

_ = modelo.fit(X = X_train, y = y_train)

Cuando utilizamos regularización, es interesante analizar cómo los coeficientes se acercan a cero cuando incrementamos alpha. 

In [ ]:
# Evolución de los coeficientes en función de alpha

alphas = modelo.alphas
coefs = []

for alpha in alphas:
  modelo_temp = Ridge(alpha=alpha, fit_intercept=False, normalize=True)
  modelo_temp.fit(X_train, y_train)
  coefs.append(modelo_temp.coef_.flatten())

fig, ax = plt.subplots(figsize=(7, 3.84))
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_xlabel('alpha')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo en función de la regularización');
plt.axis('tight')
plt.show()

In [ ]:
# Evolución del error en función de alpha
# modelo.cv_values_ almacena el mse de cv para cada valor de alpha.
# Tiene dimensiones (n_samples, n_targets, n_alphas)
mse_cv = modelo.cv_values_.reshape((-1, 200)).mean(axis=0) # mse promedio en las 150 observaciones
mse_sd = modelo.cv_values_.reshape((-1, 200)).std(axis=0)  # desvio estandar de mse

# Se aplica la raíz cuadrada para pasar de mse a rmse
rmse_cv = np.sqrt(mse_cv)
rmse_sd = np.sqrt(mse_sd)

# Se identifica el óptimo y el óptimo + 1std
min_rmse = np.min(rmse_cv)
sd_min_rmse = rmse_sd[np.argmin(rmse_cv)]
optimo = modelo.alphas[np.argmin(rmse_cv)]

# Gráfico del error +- 1 desviación estándar
fig, ax = plt.subplots(figsize=(7, 3.84))
ax.plot(modelo.alphas, rmse_cv)
ax.fill_between(
    modelo.alphas,
    rmse_cv + rmse_sd,
    rmse_cv - rmse_sd,
    alpha=0.2
)

ax.axvline(
    x=optimo,
    c="gray",
    linestyle='--',
    label='óptimo'
)

ax.set_xscale('log')
ax.set_ylim([0, None])
ax.set_title('Evolución del error CV en función de la regularización')
ax.set_xlabel('alpha')
ax.set_ylabel('RMSE')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Mejor valor alpha encontrado
print(f"Mejor valor de alpha encontrado: {modelo.alpha_}")

In [ ]:
# Predicciones test
y_pred = modelo.predict(X=X_test)
y_pred = y_pred.flatten()

# Error de test del modelo 
rmse_ridge = mean_squared_error(
    y_true = y_test,
    y_pred = y_pred,
    squared = False
)

print("")
print(f"El error (rmse) de test es: {rmse_ridge}")

### Lasso

La regularización Lasso penaliza la suma del valor absolutos de los coeficientes de regresión. A esta penalización se le conoce como l1 y tiene el efecto de forzar a que los coeficientes de los predictores tiendan a cero. Dado que un predictor con coeficiente de regresión cero no influye en el modelo, lasso consigue excluir los predictores menos relevantes. El grado de penalización está controlado por el hiperparámetro alpha. Cuando alpha=0 el resultado es equivalente al de un modelo lineal por mínimos cuadrados ordinarios. A medida que alpha aumenta, mayor es la penalización y más predictores quedan excluidos.

In [ ]:
# Creación y entrenamiento del modelo (con búsqueda por CV del valor óptimo alpha)
# Por defecto LassoCV utiliza el mean squared error
modelo = LassoCV(
    alphas = np.logspace(-10, 3, 200),
    normalize = True,
    cv = 10
)
_ = modelo.fit(X=X_train, y=y_train)

In [ ]:
# Evolución de los coeficientes en función de alpha
alphas = modelo.alphas_
coefs = []

for alpha in alphas:
    modelo_temp = Lasso(alpha=alpha, fit_intercept=False, normalize=True)
    modelo_temp.fit(X_train, y_train)
    coefs.append(modelo_temp.coef_.flatten())

fig, ax = plt.subplots(figsize=(7, 3.84))
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_ylim([-15,None])
ax.set_xlabel('alpha')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo en función de la regularización');

In [ ]:
# Número de predictores incluidos (coeficiente !=0) en función de alpha
alphas = modelo.alphas_
n_predictores = []

for alpha in alphas:
    modelo_temp = Lasso(alpha=alpha, fit_intercept=False, normalize=True)
    modelo_temp.fit(X_train, y_train)
    coef_no_cero = np.sum(modelo_temp.coef_.flatten() != 0)
    n_predictores.append(coef_no_cero)

fig, ax = plt.subplots(figsize=(7, 3.84))
ax.plot(alphas, n_predictores)
ax.set_xscale('log')
ax.set_ylim([-15,None])
ax.set_xlabel('alpha')
ax.set_ylabel('nº predictores')
ax.set_title('Predictores incluidos en función de la regularización');

In [ ]:
# Evolución del error en función de alpha
# modelo.mse_path_ almacena el mse de cv para cada valor de alpha
# Tiene dimensiones (n_alphas, n_folds)
mse_cv = modelo.mse_path_.mean(axis=1)
mse_sd = modelo.mse_path_.std(axis=1)

# Se aplica la raíz cuadrada para pasar de mse a rmse
rmse_cv = np.sqrt(mse_cv)
rmse_sd = np.sqrt(mse_sd)

# Se identifica el óptimo y el óptimo + 1std
min_rmse     = np.min(rmse_cv)
sd_min_rmse  = rmse_sd[np.argmin(rmse_cv)]
optimo       = modelo.alphas_[np.argmin(rmse_cv)]

# Gráfico del error +- 1 desviación estándar
fig, ax = plt.subplots(figsize=(7, 3.84))
ax.plot(modelo.alphas_, rmse_cv)
ax.fill_between(
    modelo.alphas_,
    rmse_cv + rmse_sd,
    rmse_cv - rmse_sd,
    alpha=0.2
)

ax.axvline(
    x = optimo,
    c = "gray",
    linestyle = '--',
    label = 'óptimo'
)

ax.set_xscale('log')
ax.set_ylim([0,None])
ax.set_title('Evolución del error CV en función de la regularización')
ax.set_xlabel('alpha')
ax.set_ylabel('RMSE')
plt.legend();

In [ ]:
# Mejor valor alpha encontrado
print(f"Mejor valor de alpha encontrado: {modelo.alpha_}")

In [ ]:
y_pred = modelo.predict(X=X_test)
y_pred = y_pred.flatten()

# Error de test del modelo 
rmse_lasso = mean_squared_error(
    y_true = y_test,
    y_pred = y_pred,
    squared = False
)

print("")
print(f"El error (rmse) de test es: {rmse_lasso}")

### Elastic Net

In [ ]:
# Creación y entrenamiento del modelo (con búsqueda por CV del valor óptimo alpha)
# Por defecto ElasticNetCV utiliza el mean squared error
modelo = ElasticNetCV(
    l1_ratio = [0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99],
    alphas = np.logspace(-10, 3, 200),
    normalize = True,
    cv = 10
)

_ = modelo.fit(X = X_train, y = y_train)

In [ ]:
# Evolución del error en función de alpha y l1_ratio
# modelo.mse_path_ almacena el mse de cv para cada valor de alpha y l1_ratio.
# Tiene dimensiones (n_l1_ratio, n_alpha, n_folds)

# Error medio de las 10 particiones por cada valor de alpha y l1_ratio 
mean_error_cv = modelo.mse_path_.mean(axis =2)

# El resultado es un array de dimensiones (n_l1_ratio, n_alpha)
# Se convierte en un dataframe
df_resultados_cv = pd.DataFrame(
                        data   = mean_error_cv.flatten(),
                        index  = pd.MultiIndex.from_product(
                                    iterables = [modelo.l1_ratio, modelo.alphas_],
                                    names     = ['l1_ratio', 'modelo.alphas_']
                                 ),
                        columns = ["mse_cv"]
                    )

df_resultados_cv['rmse_cv'] = np.sqrt(df_resultados_cv['mse_cv'])
df_resultados_cv = df_resultados_cv.reset_index().sort_values('mse_cv', ascending = True)
df_resultados_cv